In [1]:
import csv
import cv2
import numpy as np

samples = []
with open('./data/data/driving_log.csv') as csvFile:
    reader = csv.reader(csvFile)
    for line in reader:
        samples.append(line)

In [2]:
from sklearn.model_selection import train_test_split
train_valid, test = train_test_split(samples, test_size=0.01)
train_samples, validation_samples = train_test_split(train_valid, test_size=0.1)


In [13]:
import cv2
import numpy as np
import sklearn
correction = [0.0,0.20,-0.20]

def generator(samples, batch_size=32):
    num_samples = len(samples)
    print("num of samples = " + str(num_samples))
    while 1: # Loop forever so the generator never terminates
#         shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                for idx in range(3):
                    name = './data/data/IMG/'+batch_sample[idx].split('/')[-1]
                    image = cv2.imread(name)
                    vimage=image.copy()
                    vimage=cv2.flip(image,1)
                    images.append(image)
                    images.append(vimage)
#                     print(batch_samples)
                    angle = float(batch_sample[3]) + correction[idx]
                    angles.append(angle)
                    angles.append(-angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=64)
validation_generator = generator(validation_samples, batch_size=64)

In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10926888446949575669
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1492402176
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7691141068079484390
physical_device_desc: "device: 0, name: Quadro M1000M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [16]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, MaxPooling2D, Cropping2D, Dropout
import tensorflow as tf

with tf.device('/gpu:0'):
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))
    model.add(Conv2D(8,(5,5),activation="relu"))
    model.add(MaxPooling2D())
    model.add(Conv2D(16,(5,5),activation="relu"))
    model.add(MaxPooling2D())
    model.add(Conv2D(32,(5,5),activation="relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(120,activation="relu"))
    model.add(Dense(84,activation="relu"))
    model.add(Dense(1))

    model.compile(loss='mse', optimizer='adam')
    print(len(train_samples))
    train_samples_per_epoch = len(train_samples)*6
    valid_samples_per_epoch = len(validation_samples)*6
    model.fit_generator(train_generator, steps_per_epoch=train_samples_per_epoch,
                        validation_data=validation_generator, 
                        validation_steps=valid_samples_per_epoch, epochs=1)
model.save('model.h5')
exit()



7159
num of samples = 796
num of samples = 7159Epoch 1/1

   39/42954 [..............................] - ETA: 8:25:06 - loss: 3.6110

KeyboardInterrupt: 